##Getting the dataset and libraries

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
from scipy import stats
import datetime as dt
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse

##Date Time Formatting

In [0]:
data = pd.read_excel('AirQualityUCI.xlsx')
data = data.drop(['T','RH','AH'],axis=1)

encoder = preprocessing.LabelEncoder()

data['Day']= pd.to_datetime(data['Date'],format = '%Y:%m:%d').dt.day_name()
data['month']= pd.to_datetime(data['Date'],format = '%Y:%m:%d').dt.month
data['Day'] = encoder.fit_transform(data['Day'].astype(str))
data['Time'] = pd.to_datetime(data['Time'], format='%H:%M:%S').dt.hour
print(data)


##Missing Values and Outliers Formatting

In [0]:
pollutants = data[['CO(GT)', 'PT08.S1(CO)', 'NMHC(GT)', 'C6H6(GT)',
       'PT08.S2(NMHC)', 'NOx(GT)', 'PT08.S3(NOx)', 'NO2(GT)', 'PT08.S4(NO2)',
       'PT08.S5(O3)']]
pollutants = pollutants.replace(to_replace=-200,value=np.nan)

for column in pollutants.columns:
  dat = data['{}'.format(column)]
  z = np.abs(stats.zscore(dat))   #Detecting outliers
  dat = dat[(z<3)]
  data['{}'.format(column)] = dat
  data['{}'.format(column)] = pollutants['{}'.format(column)].replace(to_replace=np.nan,value=np.mean(pollutants['{}'.format(column)]))
  
data = data.drop(['Date'],axis=1)

##Dataset Plotting

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.bar(data['month'],data['CO(GT)'],color='g')
plt.title('Monthly CO Conc.')
plt.xlabel('Month')
plt.ylabel('CO Conc.')
x= ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
plt.xticks(range(1,13),x)
plt.show()

plt.figure(figsize = (13,13))
sns.heatmap(data.corr(),annot=True,cmap='viridis')
plt.title('Heatmap of co-relation between variables',fontsize=12)
plt.show()

"import matplotlib.pyplot as plt\nimport seaborn as sns\n\nplt.bar(data['month'],data['CO(GT)'],color='g')\nplt.title('Monthly CO Conc.')\nplt.xlabel('Month')\nplt.ylabel('CO Conc.')\nx= ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']\nplt.xticks(range(1,13),x)\nplt.show()\n\nplt.figure(figsize = (13,13))\nsns.heatmap(data.corr(),annot=True,cmap='viridis')\nplt.title('Heatmap of co-relation between variables',fontsize=12)\nplt.show()"

## Testing Assumptions

####Winter Split

In [0]:
win1 = data[(data['month']<=2)]
win2 = data[(data['month']>=11)]
winter = pd.concat([win1,win2],axis=0,join='outer')

com_X = winter[['Time','month','Day']]

"win1 = data[(data['month']<=2)]\nwin2 = data[(data['month']>=11)]\nwinter = pd.concat([win1,win2],axis=0,join='outer')\n\ncom_X = winter[['Time','month','Day']]"

###Dataset Split (Entire Year)

In [0]:
X1 = data[['Time','month','Day']]
Y1 = data['CO(GT)']

###Various test set sizes

In [0]:
test_size = list(np.linspace(0.1,0.9,9,endpoint=True))
coef = []
error = []

for size in test_size:
    X_train1, X_test1, Y_train1, Y_test1 = train_test_split(X1,Y1,test_size=size,random_state=19)
    regressor.fit(X_train1,np.array(Y_train1).reshape(Y_train1.shape[0],1))
    pred1 = np.array(regressor.predict(X_test1))

    cod1 = regressor.score(X_test1,np.array(Y_test1).reshape(Y_test1.shape[0],1))
    ms1 = mse(pred1,np.array(Y_test1).reshape(Y_test1.shape[0],1))

    error.append(ms1)
    coef.append(cod1)

In [0]:
import matplotlib.pyplot as plt

plt.plot(test_size,error,color='r',marker='o')
plt.xlabel('Test Size')
plt.ylabel('MSE')
plt.title('Mean Squared Error for various test sizes')
plt.show()

### End of previous module

In [0]:
X_train1, X_test1, Y_train1, Y_test1 = train_test_split(X1,Y1,test_size=0.1,random_state=19)
regressor.fit(X_train1,np.array(Y_train1).reshape(Y_train1.shape[0],1))
pred1 = np.array(regressor.predict(X_test1))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [0]:
ma1 = mae(pred1,np.array(Y_test1).reshape(Y_test1.shape[0],1))
cod1 = regressor.score(X_test1,np.array(Y_test1).reshape(Y_test1.shape[0],1))
ms1 = mse(pred1,np.array(Y_test1).reshape(Y_test1.shape[0],1))

In [0]:
print('*** Evaluation metrics for CO ***\n')
print('Coeff. of Determination: ',cod1)
print('Mean Squared Error: ',ms1)
print('Root Mean Sqaured Error: ',ms1**0.5)
print('Mean Absolute Error: ',ma1)
a = pd.DataFrame(Y_test1)
a['pred']= regressor.predict(X_test1)
print('\n\tTable 1\n')
print(a.head())

##Applying Grid Search CV

In [0]:
X1 = data[['Time','month','Day']]
Y1 = data['CO(GT)']
regressor = RandomForestRegressor()

n_estimators = [100,200,300,400,500]
max_depth = [None,80,100,120,150]
min_samples_split = [6,7,8,9,10]
min_samples_leaf = [3,4,5,6]

params_grid = {'n_estimators':n_estimators,'max_depth':max_depth,'min_samples_split':min_samples_split,'min_samples_leaf':min_samples_leaf}

gridder = GridSearchCV(estimator=regressor,param_grid=params_grid,n_jobs=-1,cv=5,verbose=5 )
gridder.fit(X1,np.array(Y1).reshape(Y1.shape[0],1))

Fitting 5 folds for each of 500 candidates, totalling 2500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   12.3s
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 284 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:  7.6min
[Parallel(n_jobs=-1)]: Done 644 tasks      | elapsed: 11.0min
[Parallel(n_jobs=-1)]: Done 878 tasks      | elapsed: 14.9min
[Parallel(n_jobs=-1)]: Done 1148 tasks      | elapsed: 19.4min
[Parallel(n_jobs=-1)]: Done 1454 tasks      | elapsed: 24.4min
[Parallel(n_jobs=-1)]: Done 1796 tasks      | elapsed: 30.1min
[Parallel(n_jobs=-1)]: Do

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'max_depth': [None, 80, 100, 120,

###Best Estimator :  
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=100,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=6, min_samples_split=6,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [0]:
print(gridder.best_estimator_)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=100,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=6, min_samples_split=6,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)


##Visualizing Decision Tree from RF

In [0]:
regressor = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=100, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, min_samples_leaf=6, min_samples_split=6, min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1, oob_score=False, random_state=None, verbose=5, warm_start=False)
#regressor.fit(X1,np.array(Y1).reshape(Y1.shape[0],1))

In [0]:
X = data[['Time','month','Day']]
Y = pd.DataFrame(data['CO(GT)'])

estimator = regressor.estimators_[5]
from sklearn.tree import export_graphviz
# Export as dot file
export_graphviz(estimator, out_file='tree.dot', 
                feature_names = X.columns,
                class_names = Y.columns,
                rounded = True, proportion = False, 
                precision = 2, filled = True)

# Convert to png using system command (requires Graphviz)
from subprocess import call
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

# Display in jupyter notebook
from IPython.display import Image
Image(filename = 'tree.png')

In [0]:
from sklearn.ensemble import AdaBoostRegressor
X1 = data[['month','Day','Time','PT08.S1(CO)','C6H6(GT)','PT08.S2(NMHC)','PT08.S5(O3)']]
Y1 = data['CO(GT)']
X_train1, X_test1, Y_train1, Y_test1 = train_test_split(X1,Y1,test_size=0.9,random_state=19)
ada = AdaBoostRegressor(regressor,loss='exponential',learning_rate=0.01)
ada.fit(X_train1,np.array(Y_train1).reshape(Y_train1.shape[0],1))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:    0.1s


building tree 1 of 100building tree 2 of 100

building tree 3 of 100building tree 4 of 100

building tree 5 of 100building tree 6 of 100

building tree 7 of 100building tree 8 of 100

building tree 9 of 100building tree 10 of 100

building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100building tree 26 of 100

building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
b

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  14 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  68 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 2 of 100
building tree 1 of 100
building tree 3 of 100building tree 4 of 100

building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100building tree 26 of 100

building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100building tree 38 of 100

building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
b

[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  14 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  68 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.2s finished


...

building tree 1 of 100
building tree 2 of 100
building tree 3 of 100building tree 4 of 100

building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100building tree 22 of 100

building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100building tree 34 of 100

building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100building tree 40 of 100

building tree 41 of 100
building tree 42 of 100
b

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  14 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  68 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    0.0s


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100building tree 24 of 100

building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100bu

[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  14 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  68 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finished


AdaBoostRegressor(base_estimator=RandomForestRegressor(bootstrap=True,
                                                       criterion='mse',
                                                       max_depth=100,
                                                       max_features='auto',
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=6,
                                                       min_samples_split=6,
                                                       min_weight_fraction_leaf=0.0,
                                                       n_estimators=100,
                                                       n_jobs=-1,
                                                       oob_score=False,
                          

In [0]:
pred3 = np.array(ada.predict(X_test1))

cod3 = ada.score(X_test1,np.array(Y_test1).reshape(Y_test1.shape[0],1))
ms3 = mse(pred3,np.array(Y_test1).reshape(Y_test1.shape[0],1))
ma3 = mae(pred3,np.array(Y_test1).reshape(Y_test1.shape[0],1))

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  14 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  68 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  14 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  68 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  14 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  68 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  14 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  68 tasks      |

In [0]:
print('*** Evaluation metrics for CO ***\n')
print('Coeff. of Determination: ',cod3)
print('Mean Squared Error: ',ms3)
print('Root Mean Sqaured Error: ',ms3**0.5)
print('Mean Absolute Error: ',ma3)
a = pd.DataFrame(Y_test1)
a['pred']= ada.predict(X_test1)
print('\n\tTable 3\n')
print(a.head())

*** Evaluation metrics for CO ***

Coeff. of Determination:  0.7269835267876117
Mean Squared Error:  0.47654124794540403
Root Mean Sqaured Error:  0.6903196708376519
Mean Absolute Error:  0.47576423790002415


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  14 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  68 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  14 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  68 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  14 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  68 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  14 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  68 tasks      |


	Table 3

       CO(GT)      pred
3418  2.15275  1.524120
6934  6.70000  4.783761
2196  1.20000  1.708819
5392  2.60000  2.775914
7582  3.00000  2.762826
